## Global-multiple BFNW

O algoritmo definido na seção anterior faz o alinhamento global entre duas sequências, porém, dado um conjunto $S = \{s_1, s_2, ..., s_{k-1}, s_k\}$ com k *strings*, a mesma ideia pode ser expandida e generalizada para alinhar esse conjunto simultaneamente. Isso pode ser feito ao modificar a estratégia de geração do grafo inicial para que, em vez de ter um aspecto bidimensional fixo, ele possa ser expandido para uma estrutura dinâmica k-dimensional. A partir desse novo grafo com estrutura similar a um tensor, basta executar exatamente a mesma estratégia de priorização, mas agora o caminhamento deve ser feito a partir do nó fonte $(0, 0, ..., 0, 0)_k$ até o nó sumidouro $(|s_1|, |s_2|, ..., |s_{k-1}|, |s_k|)_k$.

In [1]:
import numpy as np

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import utils.constantes as constantes
import utils.func_utils as func_utils
import utils.maxheap as maxheap

In [2]:
class NodoAuxiliar:

    def __init__(self, posicao, valor_acumulado, direcao):
        """
        posição: A posição desse nodo no tensor n-dimensional.
        
        valor_acumulado: O valor, ganho ou pontuação acumulada associada a esse nodo
        
        direcao: Vetor composto por 0's e 1's que indica qual direção foi tomada para se chegar a esse nodo
        """
        self.posicao = posicao
        self.valor_acumulado = valor_acumulado
        self.direcao = direcao

    def __str__(self):
        return "Posição: " + str(self.posicao) + ", va: " + str(self.valor_acumulado) + ", dir: " + str(self.direcao)

    def __repr__(self):
        return str(self)

    def __lt__(self, other):
        return self.valor_acumulado < other.valor_acumulado

    def __le__(self, other):
        return self.valor_acumulado <= other.valor_acumulado

    def __gt__(self, other):
        return self.valor_acumulado > other.valor_acumulado

    def __ge__(self, other):
        return self.valor_acumulado >= other.valor_acumulado

In [3]:
def gerar_permutacoes_direcoes(num_seqs, aux, direcoes, index):
    """
    Observação IMPORTANTE: Assim como no bfnw original, a ordem nas quais essas permutações
    são geradas INFLUENCIAM no caminhamento final, no sentido de que quais direções são
    testadas primeiro podem alterar por completo o caminhamento e resultado final
    """
    aux[index] = 0
    
    if (index == num_seqs-1):
        direcoes.append(aux.copy())
    else:
        gerar_permutacoes_direcoes(num_seqs, aux, direcoes, index+1)
    
    aux[index] = 1
    
    if (index == num_seqs-1):
        direcoes.append(aux.copy())
    else:
        gerar_permutacoes_direcoes(num_seqs, aux, direcoes, index+1)
    
def global_multiple_bfnw(
    sequencias,
    matriz_pontuacao=constantes.matriz_pontuacao_blosum62,
    dicionario_indice_alfabeto=constantes.dicionario_indice_alfabeto_all_amino,
    penalidade_indel=constantes.PENALIDADE_INDEL):
    
    num_seqs = len(sequencias)
    tam_seqs = [ len(s) for s in sequencias ]
    
    mapa_posicionamento = {}
    mapa_similaridade = {}
    # TODO: Substituir por matriz de n dimensões e ver se dá na mesma
    
    # Gerando todas as permutações de todas as "direções" possíveis para as arestas
    # desse problema. Isso varia dependendo da qtde de sequências/dimensões.
    direcoes = []
    gerar_permutacoes_direcoes(num_seqs, [0] * num_seqs, direcoes, 0)
    direcoes.pop(0) # Removendo a permutação [0,0,0,...0], pois não faz sentido inserir gap em todas as seqs
    #direcoes = [[1, 0], [0, 1], [1, 1]]
    
    fonte = NodoAuxiliar([0] * num_seqs, 0, [-np.inf] * num_seqs)
    heap = maxheap.MaxHeap()
    heap.insert(fonte)
    
    while not heap.isEmpty():
    
        # Buscando o melhor vértice ou nó ainda aberto "k" (nesse algorítmo, o com maior valor)
        k = heap.extractMax()

        # Fechando esse nó:
        tupla_posicao = tuple(k.posicao)
        if (tupla_posicao not in mapa_similaridade) or k.valor_acumulado > mapa_similaridade[tupla_posicao]:
            mapa_similaridade[tupla_posicao] = k.valor_acumulado
            mapa_posicionamento[tupla_posicao] = k.direcao
        else:
            continue
        
        # Verificando se chegou no vértice sumidouro!!!
        # TODO: Uma forma de se verificar isso com tradeoff entre tempo e memória (não precisa
        # verificar o vetor posicao abaixo, mas exige uma variável auxiliar) seria se compararmos
        # a soma de todas as dimensões da posição desse k com sum(tam_seqs)
        if k.posicao == tam_seqs:
            break
        
        # Computando os próximos vértices
        for direcao in direcoes:
            # A nova posição para esse vértice é a posição de k somada à direção dessa iteração
            nova_posicao = [0] * num_seqs
            posicao_invalida = False
            for i in range(0, num_seqs):
                nova_posicao[i] = k.posicao[i] + direcao[i]

                # Nem todas as novas posições são válidas
                if nova_posicao[i] > tam_seqs[i]:
                    posicao_invalida = True
                    break

            if posicao_invalida:
                continue

            # O novo valor para esse vértice é o valor de k acumulado a todas as combinações de
            # matches que possam ter ocorrido (combinação de "1"s no vetor de direção) e
            # acumulado ainda a todos os indels que foram realizados
            novo_valor_acumulado = k.valor_acumulado
            for i in range(0, num_seqs):
                if direcao[i]:
                    # Se i quer fazer match, encontre quem em j também quer fazer match:
                    for j in range(i+1, num_seqs):
                        if direcao[j]:
                            #print(i, "-", j, "   -->   " , (sequencias[i])[k.posicao[i]], "-", (sequencias[j])[k.posicao[j]])
                            novo_valor_acumulado += matriz_pontuacao[dicionario_indice_alfabeto[(sequencias[i])[k.posicao[i]]], dicionario_indice_alfabeto[(sequencias[j])[k.posicao[j]]]]
                else:
                    # Se tem gap em i, aplique a penalidade de indel
                    novo_valor_acumulado -= penalidade_indel

            heap.insert(NodoAuxiliar(nova_posicao, novo_valor_acumulado, direcao.copy()))

    return mapa_posicionamento, mapa_similaridade

In [4]:
def computar_alinhamento(sequencias, mapa_posicionamento):
    num_seqs = len(sequencias)
    tam_seqs = [ len(s) for s in sequencias ]
    
    alinhamento = [""] * num_seqs

    contadores_seq = tam_seqs.copy()
    
    all_zeros = [0] * num_seqs

    while contadores_seq > all_zeros:
        
        tupla_contadores_seq = tuple(contadores_seq)
        
        direcao = mapa_posicionamento[tupla_contadores_seq]
        
        for (k, d) in enumerate(direcao):
            if d == 1:
                alinhamento[k] += (sequencias[k])[contadores_seq[k]-1]
                contadores_seq[k] -= 1
            else:
                alinhamento[k] += "-"

    return [ s[::-1] for s in alinhamento ]

In [5]:
def printar_valores(sequencias, mapa_posicionamento, mapa_similaridade):
    print("Alinhamento global múltiplo das strings a seguir:")
    for s in sequencias:
        print(s)
    print("----------------------")
    
    print("\nMaior Pontuação: ", mapa_similaridade[tuple([ len(s) for s in sequencias ])])
    
    alinhamento = computar_alinhamento(sequencias, mapa_posicionamento)
    print("\nAlinhamento")
    for a in alinhamento:
        print(a)

In [6]:
matriz_pontuacao_ATGC = np.array([ [1, 0, 0, 0],
                                   [0, 1, 0, 0],
                                   [0, 0, 1, 0],
                                   [0, 0, 0, 1]])

dicionario_indice_alfabeto_ATGC = {
    "A": 0,
    "T": 1,
    "G": 2,
    "C": 3
}

v = "ACTA"
w = "ACTC"
x = "AACC"
sequencias=[v, w, x]

mapa_posicionamento, mapa_similaridade = global_multiple_bfnw(
    matriz_pontuacao=matriz_pontuacao_ATGC,
    dicionario_indice_alfabeto=dicionario_indice_alfabeto_ATGC,
    penalidade_indel=1,
    sequencias=sequencias)

printar_valores(sequencias, mapa_posicionamento, mapa_similaridade)

Alinhamento global múltiplo das strings a seguir:
ACTA
ACTC
AACC
----------------------

Maior Pontuação:  6

Alinhamento
ACTA
ACTC
AACC


In [7]:
v="ATCTGAT"
w="TGCATA"
sequencias=[v,w]

mapa_posicionamento, mapa_similaridade = global_multiple_bfnw(
    matriz_pontuacao=matriz_pontuacao_ATGC,
    dicionario_indice_alfabeto=dicionario_indice_alfabeto_ATGC,
    penalidade_indel=0,
    sequencias=sequencias)

printar_valores(sequencias, mapa_posicionamento, mapa_similaridade)

Alinhamento global múltiplo das strings a seguir:
ATCTGAT
TGCATA
----------------------

Maior Pontuação:  3

Alinhamento
ATC-TGAT
TGCAT-A-


In [8]:
v="AATGCAGTCA"
w="ATGCTAGA"
x="ATGCTGCTCAGA"
sequencias=[v,w,x]

mapa_posicionamento, mapa_similaridade = global_multiple_bfnw(
    matriz_pontuacao=matriz_pontuacao_ATGC,
    dicionario_indice_alfabeto=dicionario_indice_alfabeto_ATGC,
    penalidade_indel=0,
    sequencias=sequencias)

printar_valores(sequencias, mapa_posicionamento, mapa_similaridade)

Alinhamento global múltiplo das strings a seguir:
AATGCAGTCA
ATGCTAGA
ATGCTGCTCAGA
----------------------

Maior Pontuação:  13

Alinhamento
A----A-TGC--AGTCA
ATGCTAGA---------
ATGCT-G--CTCAG--A


### Teste final

In [9]:
v="PDQSYNWTQGDEVTEY"
w="DQSYNWAQGDEVDEY"
x="PDQSYNQGDEVDEF"
sequencias=[v,w,x]

mapa_posicionamento, mapa_similaridade = global_multiple_bfnw(sequencias=sequencias)

printar_valores(sequencias, mapa_posicionamento, mapa_similaridade)

Alinhamento global múltiplo das strings a seguir:
PDQSYNWTQGDEVTEY
DQSYNWAQGDEVDEY
PDQSYNQGDEVDEF
----------------------

Maior Pontuação:  194

Alinhamento
PDQSYNWTQGDEV---TEY
-DQSYNWAQGDEVDEY---
PDQSYN--QGDEVDEF---


In [10]:
string_teste = func_utils.gerar_string_aleatoria(30, constantes.dicionario_indice_alfabeto_all_amino)
string_teste

'CQWKSKVKCLSPIANGVLKARPFECWVYQC'

In [11]:
sequencias_teste = [string_teste] + [func_utils.mutar_string(string_teste, constantes.dicionario_indice_alfabeto_all_amino, 20) for i in range(4)]
sequencias_teste

['CQWKSKVKCLSPIANGVLKARPFECWVYQC',
 'CQWKSKVKCGSPITTGVLKARPFVCWMYQC',
 'CQWQSKVKCFSPEANGVAKARPIECWVYQC',
 'CQWKSKVHCFEPHANGVLKARVFECWNYQC',
 'CVWKSKMKCLSNIANGVLKARPIEAWVYYC']

In [12]:
mapa_posicionamento, mapa_similaridade = global_multiple_bfnw(sequencias=sequencias_teste)

printar_valores(sequencias_teste, mapa_posicionamento, mapa_similaridade)

Alinhamento global múltiplo das strings a seguir:
CQWKSKVKCLSPIANGVLKARPFECWVYQC
CQWKSKVKCGSPITTGVLKARPFVCWMYQC
CQWQSKVKCFSPEANGVAKARPIECWVYQC
CQWKSKVHCFEPHANGVLKARVFECWNYQC
CVWKSKMKCLSNIANGVLKARPIEAWVYYC
----------------------

Maior Pontuação:  564

Alinhamento
CQWKSKVKC----L--S-----PI-AN----GVLKARP-F-E---C-WVYQC--------------------
CQWKSKVKC----------G-SPI----------------------------TTGVLKAR-PF-VCW-MYQC
CQWQSKVKCF-S--PEAN-GVA--KARPI-----E---CWV--YQC--------------------------
CQWKSKV----------HC----FE--P--H----A-----------------NGVLKARV-FE-CWN-YQC
C-----V--WKSKM-K--C-LS----N-IANGVLKARP--IEAW--VY-Y-C--------------------
